### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)


# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.


In [77]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

### Displaying shell command output:
**Note**: Will not display _asadm_ or _aql_ executables output.

In [33]:
String command = "pwd";
 
try {
    Process process = Runtime.getRuntime().exec(command);
 
    BufferedReader reader = new BufferedReader(
            new InputStreamReader(process.getInputStream()));
    String line;
    while ((line = reader.readLine()) != null) {
        System.out.println(line);
    }
 
    reader.close();
 
} catch (IOException e) {
    e.printStackTrace();
}

/home/training/student-workbook/v7.0/jn/codeExamples


# Add Java Client POM Dependency

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [4]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Accessing a record on the Aerospike Server
We build the Key object in Java
We need the namespace and the record digest to find the record on the server.
The record digest is computed by the client library using application provided key (integer, string or byte-array) and the set name. If not in a set, use null for set name.

<img src="./graphics/RecordKey.png"
     alt="Record Digest"
     style="float: left; margin-right: 10px;"
     width="600"
     height="400"/>

# Insert test records
* Our one node server is already running on localhost (same server as the Jupyter Notebook server).
* Unlike in iPython kernel for Jupyter Notebook, we can only execute a single line of shell command in IJava kernel.

In [5]:
%sh aql -c "set key_send true"

In [78]:
%sh aql -f "./aqlScripts/expTest.aql"

In [79]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Initialized the client and connected to the cluster.
Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
Read back the record.
Record values are:
(gen:1),(exp:471331922),(bins:(a:1),(b:2),(c:[c3, c4]))


# Transaction Operations
* Example record:
  * record [ a: '1', b: '2', c: '["c3", "c4"]']
 
* transactionOperations [
  * //operations that conditionally modify the record
  * write(c, expression[ c contains "c3"] ? remove "c3": do_nothing)
  * ...
  * //operations to delete the record if b is nil and list c is empty or doesn’t exist
  * write(a, expression[ b == nil && size c == 0] :  nil : do_nothing)
  * write(c, expression[ b == nil && size c == 0] :  nil : do_nothing)


In [80]:
//Needed imports
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.Exp.Type;
import com.aerospike.client.Operation;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListReturnType;
import com.aerospike.client.exp.ListExp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpWriteFlags;
import com.aerospike.client.exp.Expression;

//Record we are working on

Key key = new Key("test", "testset", "key1");

WritePolicy wPolicy = new WritePolicy();

record = client.get(null, key);
System.out.println("Read back the starting record.");
System.out.println(record);

//Remove c3 from list and check record
String testVal = "c3";

Exp expListIsEmpty = Exp.lt(ListExp.size(Exp.listBin("c")), Exp.val(1));
Expression expValListIsEmptyAndbisNil = Exp.build( 
    Exp.cond(
       Exp.and(expListIsEmpty, Exp.not(Exp.binExists("b"))), Exp.nil(), Exp.unknown()) );

Record rec = client.operate( wPolicy, key,   
          ListOperation.removeByValue("c", Value.get(testVal), ListReturnType.NONE)                  
         );

record = client.get(null, key);
System.out.println("Read back the record after removing c3 from list.");
System.out.println(record);

// Run record deletion code, sets all bins to null, if conditions allow
Record rec = client.operate( wPolicy, key,                     
          ExpOperation.write("a", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL),
          ExpOperation.write("b", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL),
          ExpOperation.write("c", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL)                  
         );

record = client.get(null, key);
System.out.println("Run conditional delete, read back the record.");
System.out.println(record);

// Remove c4 from list and check record
testVal = "c4";
rec = client.operate( wPolicy, key,   
          ListOperation.removeByValue("c", Value.get(testVal), ListReturnType.NONE)                 
         );
record = client.get(null, key);
System.out.println("Read back the record after removing c4 from list.");
System.out.println(record);

// Run record deletion code, sets all bins to null, if conditions allow
Record rec = client.operate( wPolicy, key,                     
          ExpOperation.write("a", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL),
          ExpOperation.write("b", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL),
          ExpOperation.write("c", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL)                  
         );

record = client.get(null, key);
System.out.println("Run conditional delete, read back the record.");
System.out.println(record);

//Set bin b to null and check record 
Expression bNil = Exp.build(Exp.nil());
rec = client.operate( wPolicy, key,  ExpOperation.write("b", bNil, ExpWriteFlags.ALLOW_DELETE));
record = client.get(null, key);
System.out.println("Read back the record after deleting bin b.");
System.out.println(record);

// rerun same operate code and check record.
rec = client.operate( wPolicy, key,     
          ExpOperation.write("a", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL),
          ExpOperation.write("b", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL),
          ExpOperation.write("c", expValListIsEmptyAndbisNil, ExpWriteFlags.ALLOW_DELETE|ExpWriteFlags.EVAL_NO_FAIL)                  
         );
record = client.get(null, key);
System.out.println("Run conditional delete, read back the record.");
System.out.println(record);


Read back the starting record.
(gen:1),(exp:471331922),(bins:(a:1),(b:2),(c:[c3, c4]))
Read back the record after removing c3 from list.
(gen:2),(exp:471331931),(bins:(a:1),(b:2),(c:[c4]))
Run conditional delete, read back the record.
(gen:3),(exp:471331931),(bins:(a:1),(b:2),(c:[c4]))
Read back the record after removing c4 from list.
(gen:4),(exp:471331931),(bins:(a:1),(b:2),(c:[]))
Run conditional delete, read back the record.
(gen:5),(exp:471331931),(bins:(a:1),(b:2),(c:[]))
Read back the record after deleting bin b.
(gen:6),(exp:471331931),(bins:(a:1),(c:[]))
Run conditional delete, read back the record.
null


# Cleanup

In [44]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"